# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable. 
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or 
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source:
*   Web Collection #1 Source:
*   Web Collection #2 Source:



## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [74]:
#url = 'https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FIM&QO_fu146_anzr=Nv4+Pn44vr45'
import pandas as pd

def data_parser(file):
    df = pd.read_csv(file) #converting csv to df
    df = df.drop(['DISTANCE','RAMP_TO_RAMP','AIR_TIME','DEST_STATE_NM','DEST_WAC','FREIGHT','PAYLOAD','DEST_STATE_FIPS','QUARTER','DEST_CITY_NAME','DEST_STATE_ABR','DEST_AIRPORT_SEQ_ID','REGION','MAIL','UNIQUE_CARRIER_ENTITY','UNIQUE_CARRIER_NAME','UNIQUE_CARRIER','DEST_CITY_MARKET_ID','DEST_AIRPORT_ID','ORIGIN_STATE_NM','DEST','DISTANCE_GROUP','CLASS','PASSENGERS','CARRIER','ORIGIN_AIRPORT_ID','ORIGIN','ORIGIN_AIRPORT_SEQ_ID','ORIGIN_CITY_MARKET_ID','ORIGIN_CITY_NAME','ORIGIN_WAC','ORIGIN_STATE_ABR','ORIGIN_STATE_FIPS','AIRCRAFT_TYPE','AIRCRAFT_CONFIG','AIRCRAFT_GROUP'],axis=1) #getting rid of irrelevant columns
    df = df[df['DEPARTURES_SCHEDULED'] != 0.0] #clearing rows where no departures are scheduled
    df = df[df['CARRIER_GROUP'] != 7] #carrier group 7 has no passengers according to download website
    df = df.sort_values(by='CARRIER_NAME') #sorting by the type of carrier
    df = df[df['DEPARTURES_SCHEDULED'] > df['DEPARTURES_PERFORMED']] #looking at where there were more flights scheduled than performed (when flights were cancelled)
    df.to_csv('transportation_statistics.csv', index=False) #writing to file

    return df







############ Function Call ############
data_parser('/Users/caroline/Documents/CS2316/project/download.csv')

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,AIRLINE_ID,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,YEAR,MONTH
101507,120.0,41.0,348.0,21615,"Air Charter, Inc d/b/a Air Flamenco",1,6,2024,1
74291,24.0,23.0,184.0,21615,"Air Charter, Inc d/b/a Air Flamenco",1,6,2024,2
101508,120.0,52.0,416.0,21615,"Air Charter, Inc d/b/a Air Flamenco",1,6,2024,1
95296,60.0,51.0,408.0,21615,"Air Charter, Inc d/b/a Air Flamenco",1,6,2024,3
101509,120.0,76.0,608.0,21615,"Air Charter, Inc d/b/a Air Flamenco",1,6,2024,1
...,...,...,...,...,...,...,...,...,...
62178,14.0,13.0,86.0,20333,Wright Air Service,1,5,2024,1
92524,52.0,51.0,427.0,20333,Wright Air Service,1,5,2024,2
63512,15.0,14.0,104.0,20333,Wright Air Service,1,5,2024,2
66084,17.0,16.0,116.0,20333,Wright Air Service,1,5,2024,1


## Web Collection Requirement \#1


In [75]:
import json
import requests
import pandas as pd

def web_parser1(url):
    response = requests.get(url) #getting the data
    json_data = response.json()  #parsing json
    data = json_data['data'] #getting relevant data
    columns = [col['name'] for col in json_data['meta']['view']['columns']] #columns are in the 'meta.view.columns' field

    df = pd.DataFrame(data, columns=columns) #creating dataframe
    df = df.drop(['sid','id','position','created_at','created_meta','updated_at','updated_meta','meta','DataExtractDate','ReportPeriod'],axis=1) #dropping irrelevant/ unneccesary columns
    df = df[df['FlightType'] == 'Scheduled Carriers'] #because charter and commuter flights are uninteresting to us since we are focusing on commerical flights and passengers
    df = df.sort_values(by='Domestic_International') #we want to see the difference between how many passengers have been carried by dom and int planes so we can generalize results of how different load factors would impact airlines in the long run
    df = df.reset_index(drop = True)

    df['Passenger_Count'] = df['Passenger_Count'].astype(float) #converting to float so we can use for computation
    #finding averages of dom and int passenger counts (to be used in phase 3)
    df_dom = df[df['Domestic_International']=='Domestic']
    avg_dom = df_dom['Passenger_Count'].mean().round(3)
    df_int = df[df['Domestic_International']=='International']
    avg_int = df_int['Passenger_Count'].mean().round(3)

    df.to_csv('dom_int_passenger_count', index=False) #writing new df to file


    return df



############ Function Call ############
web_parser1('https://data.lacity.org/api/views/d3a2-7j6v/rows.json?accessType=DOWNLOAD')

,Arrival_Departure,Domestic_International,FlightType,Passenger_Count
0,Arrival,Domestic,Scheduled Carriers,1128607.0
1,Arrival,Domestic,Scheduled Carriers,2419169.0
2,Departure,Domestic,Scheduled Carriers,2400638.0
3,Arrival,Domestic,Scheduled Carriers,1918767.0
4,Departure,Domestic,Scheduled Carriers,1957849.0
...,...,...,...,...
851,Departure,International,Scheduled Carriers,1055191.0
852,Departure,International,Scheduled Carriers,627113.0
853,Arrival,International,Scheduled Carriers,558794.0
854,Arrival,International,Scheduled Carriers,638866.0


## Web Collection Requirement \#2

In [76]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def web_parser2(url):
    #getting data and parsing through it
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table=soup.find('table',{'class':"wikitable"})
    #converting data to df
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    df = df.drop(['Total assets in billion US$[b]','Price per share in US$ [citation needed]','Employees[b]','Refs.'], axis=1)
    df = df.dropna(axis=0) #dropping rows with NaNs
    df = df.reset_index(drop = True) #resetting indices
    df.rename(columns={'Net income in billion US$[b]':'Net Income in Billion USD','Revenue in billion US$[a][b]':'Revenue in Billion USD','Passenger load factor (%)[b]':'Passenger Load Factor (%)','Fleet size[c]':'Fleet Size'}, inplace=True) #renaming columns to make them more readable
    #correcting the non-standard dash and converting columns to float
    df['Net Income in Billion USD'] = df['Net Income in Billion USD'].str.replace('−', '-').astype(float)


    #calculating Net Profit Margin
    df['Net Profit Margin'] = abs(df['Net Income in Billion USD'] / df['Revenue in Billion USD']) * 100 
    df.to_csv('delta_air_lines_financials.csv', index=False) #writing to file

    return df



############ Function Call ############
web_parser2('https://en.m.wikipedia.org/wiki/Delta_Air_Lines#cite_note-Airline_group-100')

/var/folders/f1/6jghkzzx3n11v1mm2rsz320c0000gn/T/ipykernel_40823/947886299.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))


,Year,Revenue in Billion USD,Net Income in Billion USD,Passenger Load Factor (%),Fleet Size,Net Profit Margin
0,2009,28.0,-1.2,82.0,983.0,4.285714
1,2010,31.7,0.6,83.0,815.0,1.892744
2,2011,35.1,0.9,82.1,775.0,2.564103
3,2012,36.6,1.0,83.8,717.0,2.732240
4,2013,37.7,10.5,83.8,743.0,27.851459
5,2014,40.3,0.7,84.7,772.0,1.736973
6,2015,40.7,4.5,84.9,809.0,11.056511
7,2016,39.6,4.3,84.6,832.0,10.858586
8,2017,41.2,3.5,85.6,856.0,8.495146
9,2018,44.4,3.9,85.5,871.0,8.783784


## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [77]:
def extra_source1(file1, file2, file3):
    df1 = pd.read_excel(file1, header=1, engine='openpyxl') #Load the file
    df1 = df1[df1['Month'] == 'TOTAL'] #Filter so the new data only shows the total load factor per year
    df1 = df1.drop(['Month'], axis = 1) #Drop the Month column

    df2 = pd.read_excel(file2, header=1, engine='openpyxl') #Load the file
    df2 = df2[df2['Quarter'] == 'TOTAL'] #Filter so the new data only shows the total net income per year
    df2 = df2.drop(['LATIN AMERICA', 'ATLANTIC', 'PACIFIC', 'Quarter'], axis = 1) #Drop the unnecessary columns

    df3 = pd.read_excel(file3, header=1, engine='openpyxl') #Load the file
    df3 = df3[df3['Quarter'] == 'TOTAL'] #Filter so the new data only shows the total operating revenue per year
    df3 = df3.drop(['LATIN AMERICA', 'ATLANTIC', 'PACIFIC', 'Quarter'], axis = 1) #Drop the unnecessary columns

    #Merge each data set by year
    merged_df = pd.merge(df1, df2, on='Year', how='outer', suffixes=(' Load Factor', ' Net Income'))

    df3 = df3.add_suffix(' Operating Revenue')
    df3.rename(columns={'Year Operating Revenue': 'Year'}, inplace=True)

    merged_df = pd.merge(merged_df, df3, on='Year', how='outer')

    #Drop rows with inconsistent data
    years_to_delete = [2000, 2001, 2002, 2024]
    indices_to_drop = merged_df[merged_df['Year'].isin(years_to_delete)].index
    merged_df = merged_df.drop(indices_to_drop)

    #Save as a new CSV file
    merged_df.to_csv('Revenue and Income VS Load Factor.csv', index=False)

    return merged_df







############ Function Call ############
fileA = '/Users/caroline/Documents/CS2316/project/LoadFactor.xlsx'
fileB = '/Users/caroline/Documents/CS2316/project/OperatingPL.xlsx'
fileC = '/Users/caroline/Documents/CS2316/project/OperatingRev.xlsx'
extra_source1(fileA,fileB,fileC)

,Year,DOMESTIC Load Factor,INTERNATIONAL Load Factor,TOTAL Load Factor,DOMESTIC Net Income,INTERNATIONAL Net Income,TOTAL Net Income,DOMESTIC Operating Revenue,INTERNATIONAL Operating Revenue,TOTAL Operating Revenue
3,2003,72.68,74.84,73.59,-1469581.0,3543.0,-1715469.0,88870096.0,1793391.0,117768014.0
4,2004,74.46,77.81,75.89,-9906154.0,151897.0,-9104424.0,100902510.0,1990252.0,134660284.0
5,2005,77.16,78.67,77.82,-19747536.0,190916.0,-27219896.0,111858063.0,2884934.0,151544403.0
6,2006,79.11,78.61,78.88,9043659.0,138042.0,18186112.0,120906925.0,2887401.0,165531803.0
7,2007,79.87,79.13,79.53,2995875.0,190398.0,7691404.0,124502978.0,3330852.0,174696416.0
8,2008,79.74,77.54,78.69,-18173970.0,-221582.0,-23749834.0,129728330.0,3941471.0,186087333.0
9,2009,81.06,78.28,79.72,-2252074.0,169340.0,-2526277.0,109681017.0,3252719.0,155051198.0
10,2010,82.18,81.59,81.90,1176134.0,304122.0,3665995.0,119073502.0,4393008.0,174677395.0
11,2011,82.87,80.30,81.60,1440558.0,-15962.0,1392188.0,132822096.0,4510153.0,193040699.0
12,2012,83.36,81.69,82.53,2212176.0,-193468.0,363732.0,135830212.0,3976258.0,196105250.0


In [ ]:
# Define further extra source functions as necessary

#Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. For the html dataset, when we converted our negative net income values to floats, the '-' symbol was not reading as a negative. We noticed this because we were getting an error when trying to create a new column using division, saying that we could not divide with type string when we had converted our numbers to floats. To fix this edge case inconsistency, we replaced the '-' in the negative values with negative signs as type float. 

2. For 2005-2008 in the html dataset, there were some/ all values of Passenger Load Factor and Fleet Size that were NaN. Because there were missing values for these years that we could not find data for, we deleted the rows for these years from the data.

3. In the csv dataset, there were many inputs with empty values, not of type NaN but of float value 0.00. We noticed this because both the 'DEPARTURES_SCHEDULED' and 'DEPARTURES_PERFORMED' columns had values of 0.00, as well as other columns in the corresponding rows. We decided to fix this by deleting only the columns where there were 0 departures scheduled, because that is not interesting to us, whereas it is important if there were flights scheduled but none performed.

4. When combining the datasets for the additional sources, we noticed that two of the datasets included information relevant to specific regions (ex. Latin America), where the third only had data that was sorted by domestic and international flights. To fix this and make the data so that it could be concatenated without any NaN values, we dropped the regional columns from the sets and just kept data in terms of domestic and international flights.

5. In the additional sources, 2002 and 2004 only had data for three out of the twelve months of the year where the other years in the dataset had information for all twelve months and a total based off of it. Since we noticed that not all months were represented for these two years, we removed them from the data because their totals were not representing the data for the full year like those of the other years in the file.
